# Various Classification Models

In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
df1 = pd.read_csv('./data/Combined_EMG_data1.csv')
df2 = pd.read_csv('./data/Combined_EMG_data2.csv')
df3 = pd.read_csv('./data/Combined_EMG_data3.csv')
df4 = pd.read_csv('./data/Combined_EMG_data4.csv')
df5 = pd.read_csv('./data/Combined_EMG_data5.csv')
df6 = pd.read_csv('./data/Combined_EMG_data6.csv')
df7 = pd.read_csv('./data/Combined_EMG_data7.csv')
df8 = pd.read_csv('./data/Combined_EMG_data8.csv')
df9 = pd.read_csv('./data/Combined_EMG_data9.csv')
df10 = pd.read_csv('./data/Combined_EMG_data10.csv')
df1.head()

,EMG Sensor 1,EMG Sensor 2,EMG Sensor 3,EMG Sensor 4,Mode
0,0.35,0.07,0.11,0.29,0
1,0.26,0.07,0.10,0.32,0
2,0.21,0.07,0.11,0.28,0
3,0.23,0.07,0.13,0.33,0
4,0.86,0.07,0.25,0.49,1


In [5]:
# separate the features and the labels into X and y variables
X1 = df1[['EMG Sensor 1', 'EMG Sensor 2', 'EMG Sensor 3', 'EMG Sensor 4']].to_numpy().reshape(-1,4)
X2 = df2[['EMG Sensor 1', 'EMG Sensor 2', 'EMG Sensor 3', 'EMG Sensor 4']].to_numpy().reshape(-1,4)
X3 = df3[['EMG Sensor 1', 'EMG Sensor 2', 'EMG Sensor 3', 'EMG Sensor 4']].to_numpy().reshape(-1,4)
X4 = df4[['EMG Sensor 1', 'EMG Sensor 2', 'EMG Sensor 3', 'EMG Sensor 4']].to_numpy().reshape(-1,4)
X5 = df5[['EMG Sensor 1', 'EMG Sensor 2', 'EMG Sensor 3', 'EMG Sensor 4']].to_numpy().reshape(-1,4)
X6 = df6[['EMG Sensor 1', 'EMG Sensor 2', 'EMG Sensor 3', 'EMG Sensor 4']].to_numpy().reshape(-1,4)
X7 = df7[['EMG Sensor 1', 'EMG Sensor 2', 'EMG Sensor 3', 'EMG Sensor 4']].to_numpy().reshape(-1,4)
X8 = df8[['EMG Sensor 1', 'EMG Sensor 2', 'EMG Sensor 3', 'EMG Sensor 4']].to_numpy().reshape(-1,4)
X9 = df9[['EMG Sensor 1', 'EMG Sensor 2', 'EMG Sensor 3', 'EMG Sensor 4']].to_numpy().reshape(-1,4)
X10 = df10[['EMG Sensor 1', 'EMG Sensor 2', 'EMG Sensor 3', 'EMG Sensor 4']].to_numpy().reshape(-1,4)
y1 = df1['Mode'].to_numpy()
y2 = df2['Mode'].to_numpy()
y3 = df3['Mode'].to_numpy()
y4 = df4['Mode'].to_numpy()
y5 = df5['Mode'].to_numpy()
y6 = df6['Mode'].to_numpy()
y7 = df7['Mode'].to_numpy()
y8 = df8['Mode'].to_numpy()
y9 = df9['Mode'].to_numpy()
y10 = df10['Mode'].to_numpy()


X = np.vstack((X1, X2, X3, X4, X5, X6, X7, X8, X9, X10))
y = np.hstack((y1, y2, y3, y4, y5, y6, y7, y8, y9, y10))

# Separate the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)


## KNN

In [18]:
m = []

for i in (x+1 for x in range(0, 25)):
    KNN = KNeighborsClassifier(n_neighbors=i)
    KNN.fit(X_train,y_train)
    scores = cross_val_score(KNN, X_train, y_train, cv=5)
    print(f"K: {i} , Mean: {scores.mean()} , Std: {scores.std()}")
    m.append(scores.mean())

print(f"Max score: {max(m)}")

K: 1 , Mean: 0.8591426893716969 , Std: 0.027092766500194022
K: 2 , Mean: 0.8484674104521434 , Std: 0.03434494325474813
K: 3 , Mean: 0.8805637110980623 , Std: 0.01962963512213646
K: 4 , Mean: 0.8852025836758661 , Std: 0.02917933875981201
K: 5 , Mean: 0.8836523781561949 , Std: 0.025587049312491738
K: 6 , Mean: 0.8836523781561949 , Std: 0.035853464876435356
K: 7 , Mean: 0.8897475044039929 , Std: 0.03082605144724297
K: 8 , Mean: 0.8882325308279508 , Std: 0.025804324915465658
K: 9 , Mean: 0.8928479154433354 , Std: 0.02494567575042592
K: 10 , Mean: 0.8867058132706986 , Std: 0.03492336188238925
K: 11 , Mean: 0.8897709923664123 , Std: 0.036278418255073316
K: 12 , Mean: 0.8867058132706986 , Std: 0.033561980650484095
K: 13 , Mean: 0.8897709923664123 , Std: 0.032193472182035415
K: 14 , Mean: 0.8897592483852026 , Std: 0.031142319043034362
K: 15 , Mean: 0.8943511450381681 , Std: 0.03143147511561248
K: 16 , Mean: 0.8882207868467411 , Std: 0.029610734014391987
K: 17 , Mean: 0.8928126834997064 , Std: 

In [19]:
KNN = KNeighborsClassifier(n_neighbors=15)

KNN.fit(X_train, y_train)
predictions = KNN.predict(X_test)
probs = KNN.predict_proba(X_test)
score = KNN.score(X_test, y_test)
# print(f"X_test: \n{X_test}")
# # print(f"Predictions: \n{predictions}")
# print(f"Probabilities: \n{probs}")
print(f"K: 15")
print(f"Score: {score}")

K: 15
Score: 0.9035714285714286


In [54]:
index = np.array([0.9, 0.1, 0.4, 0.5]).reshape(-1,4)
pred = mod.predict(index)
print(pred)

middle = np.array([1.6, 0.2, 1.5, 1.6]).reshape(-1,4)
pred = mod.predict(middle)
print(pred)

fourth = np.array([1.9, 0.3, 0.6, 1.1]).reshape(-1,4)
pred = mod.predict(fourth)
print(pred)

pinky = np.array([0.6, 0.2, 0.6, 1.1]).reshape(-1,4)
pred = mod.predict(pinky)
print(pred)

[1]
[2]
[3]
[4]


## Random Forest

In [10]:
RFC = RandomForestClassifier()

values = range(1,51)

parameter_grid = {'n_estimators': values, 'max_depth': values}

grid = GridSearchCV(estimator=RFC, param_grid=parameter_grid)

grid.fit(X_train, y_train)


GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': range(1, 51),
                         'n_estimators': range(1, 51)})

In [11]:
print(grid.best_params_)

{'max_depth': 5, 'n_estimators': 35}


In [13]:
RFC = RandomForestClassifier(n_estimators=35, max_depth=5)

RFC.fit(X_train, y_train)

y_pred = RFC.predict(X_test)

print(f"Features importances: \n{RFC.feature_importances_}")
print(f"Confusion matrix: \n{confusion_matrix(y_test, y_pred)}") # predicted vertical, actual horizontal
print(f"Score: {RFC.score(X_test, y_test)}")

Features importances: 
[0.34661151 0.09006318 0.25028285 0.31304246]
Confusion matrix: 
[[144   0   0   0   4]
 [  1  33   0   0   0]
 [  0   0  28   5   2]
 [  0   2   5  22   2]
 [  0   1   1   2  28]]
Score: 0.9107142857142857


In [55]:
index = np.array([0.9, 0.1, 0.4, 0.5]).reshape(-1,4)
pred = RFC.predict(index)
print(pred)

middle = np.array([1.6, 0.2, 1.5, 1.6]).reshape(-1,4)
pred = RFC.predict(middle)
print(pred)

fourth = np.array([1.9, 0.3, 0.6, 1.1]).reshape(-1,4)
pred = RFC.predict(fourth)
print(pred)

pinky = np.array([0.6, 0.2, 0.6, 1.1]).reshape(-1,4)
pred = RFC.predict(pinky)
print(pred)

[1]
[2]
[3]
[4]


## Multilayer Perceptron

In [6]:
MLP = MLPClassifier(hidden_layer_sizes=(200,), activation='relu', solver='adam', random_state=1, max_iter=800)
MLP.fit(X_train, y_train)

probs = MLP.predict_proba(X_test)
preds = MLP.predict(X_test)
score = MLP.score(X_test, y_test)

print(f"Score: {score}")

Score: 0.9357142857142857


In [7]:
index = np.array([0.9, 0.1, 0.4, 0.5]).reshape(-1,4)
pred = MLP.predict(index)
print(pred)

middle = np.array([1.6, 0.2, 1.5, 1.6]).reshape(-1,4)
pred = MLP.predict(middle)
print(pred)

fourth = np.array([1.9, 0.3, 0.6, 1.1]).reshape(-1,4)
pred = MLP.predict(fourth)
print(pred)

pinky = np.array([0.6, 0.2, 0.6, 1.1]).reshape(-1,4)
pred = MLP.predict(pinky)
print(pred)

[1]
[2]
[3]
[4]


### True Test Values

In [12]:
index = np.array([1.07, 0.08, 0.49, 0.61]).reshape(-1,4)
pred = MLP.predict(index)
print(pred)

middle = np.array([2.02, 0.12, 0.76, 1.42]).reshape(-1,4)
pred = MLP.predict(middle)
print(pred)

fourth = np.array([1.06, 0.16, 0.61, 1.25]).reshape(-1,4)
pred = MLP.predict(fourth)
print(pred)

pinky = np.array([0.55, 0.1, 0.28, 0.88]).reshape(-1,4)
pred = MLP.predict(pinky)
print(pred)

[1]
[2]
[3]
[4]
